In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [1]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [3]:
# 更新paddlehub版本
# !pip uninstall paddlepaddle
# !pip install paddlepaddle -i https://pypi.tuna.tsinghua.edu.cn/simple
# !pip uninstall paddlehub
# !pip install paddlehub -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 212 kB 4.9 MB/s            
     |████████████████████████████████| 100 kB 8.0 MB/s            
     |████████████████████████████████| 12.2 MB 726 kB/s            
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.


## 1. 查看项目中Paddle、PaddleHub版本及模型信息。

In [1]:
import paddlehub as hub
import paddle

p_v = paddle.__version__
hub_v = hub.__version__
print("Paddle version:",p_v+"\nPaddleHub version:", hub_v+"")
print("https://www.paddlepaddle.org.cn/hublist 这里可以查看模型信息")

Paddle version: 2.2.2
PaddleHub version: 2.2.0
https://www.paddlepaddle.org.cn/hublist 这里可以查看模型信息


## 2. 完成1个一键预测任务。

2.1 下面这个是人脸识别，但未知原因不能生成图片结果。所以再做一个文本情感分析。

In [2]:
import cv2
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg 

model1 = hub.Module(name="pyramidbox_face_detection")
res1 = model1.face_detection(data = {"image": ["zhaoliying.jpeg"]}, output_dir = './detection_output/')
print("这里我试了很多次不知为什么就是不生成预测结果到./detection_output文件夹，所以没法展示预测结果了。")

for result in res1:   
    print(result)
# img = mpimg.imread("./detection_output/zhaoliying.jpeg")
# plt.imshow(img) 
# plt.axis('off') 
# plt.show()

[2022-03-26 17:52:36,243] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object
W0326 17:52:36.247887  4339 analysis_predictor.cc:1350] Deprecated. Please use CreatePredictor instead.


这里我试了很多次不知为什么就是不生成预测结果到./detection_output文件夹，所以没法展示预测结果了。
{'data': [{'left': 195.93939208984375, 'top': 120.91435241699219, 'right': 395.1234436035156, 'bottom': 385.98822021484375, 'confidence': 0.9999722242355347}], 'path': 'zhaoliying.jpeg'}


2.2 下面这个是文本审核一键预测。

In [18]:
porn_detection_gru = hub.Module(name="porn_detection_gru")
res2 = porn_detection_gru.detection(data={"text": ["日本苍井空"]})
for result in res2:
    print(result)

[2022-03-26 17:56:34,655] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object


{'text': '日本苍井空', 'porn_detection_label': 1, 'porn_detection_key': 'porn', 'porn_probs': 0.7168, 'not_porn_probs': 0.2832}


## 3. 完成1个迁移学习任务。
//训练时间太长

In [2]:
import paddlehub.vision.transforms as T
from paddlehub.datasets import Canvas

# 数据预处理方式
transform = T.Compose([T.Resize((256, 256), interpolation='NEAREST'),
                        T.RandomPaddingCrop(crop_size=176),
                        T.RGB2LAB()], to_rgb=True)

# 获取并处理数据集
color_set = Canvas(transform=transform, mode='train')

# 加载(将被训练的)预训练模型
model = hub.Module(name='user_guided_colorization', load_checkpoint=None)
model.set_config(classification=True, prob=1)

# 设置优化策略、运行配置、训练模型
optimizer = paddle.optimizer.Adam(learning_rate=0.0001, parameters=model.parameters())
trainer = hub.Trainer(model, optimizer, checkpoint_dir='img_colorization_ckpt_cls_1')
trainer.train(color_set, epochs=100, batch_size=10, eval_dataset=color_set, log_interval=10, save_interval=10)

# 模型预测
if __name__ == '__main__':
    model = hub.Module(name='user_guided_colorization', load_checkpoint='/PATH/TO/CHECKPOINT') # 读取预训练模型
    model.set_config(prob=0.1)
    result = model.predict(images=['house.png'])



load pretrained checkpoint success


[2022-03-26 18:43:11,368] [ WARNING] - PaddleHub model checkpoint not found, start from scratch...

import numpy as np
from paddle.io import DataLoader, Dataset

class RandomDataset(Dataset):
    def __getitem__(self, idx):
        data = np.random.random((2, 3)).astype('float32')

        return data

    def __len__(self):
        return 10

dataset = RandomDataset()
loader = DataLoader(dataset, batch_size=1)
data = next(loader())

In Paddle <= 2.0, data is in format '[Tensor(shape=(1, 2, 3), dtype=float32)]', and in Paddle >= 2.1, data is in format 'Tensor(shape=(1, 2, 3), dtype=float32)'

[2022-03-26 18:48:54,727] [   TRAIN] - Epoch=1/100, Step=10/146 loss=4.7411 lr=0.000100 step/sec=0.03 | ETA 139:15:00
[2022-03-26 18:54:39,837] [   TRAIN] - Epoch=1/100, Step=20/146 loss=4.5503 lr=0.000100 step/sec=0.03 | ETA 139:36:20
[2022-03-26 19:00:24,829] [   TRAIN] - Epoch=1/100, Step=30/146 loss=4.1443 lr=0.000100 step/sec=0.03 | ETA 139:42:29
[2022-03-26 19:06:12,647] [   TRAIN] - Epoch=1

KeyboardInterrupt: 

## 4. 完成1个demo服务化部署任务。

In [5]:
#LAC分词服务部署

# 在终端启动Paddlehub Serving
# $ hub serving start -m lac

import requests
import json

# 部署lac分词服务
if __name__ == "__main__":
    # 指定用于预测的文本并生成字典{"text": [text_1, text_2, ... ]}
    text = ["今天是个好日子", "天气预报说今天要下雨"]
    # 以key的方式指定text传入预测方法的时的参数，此例中为"data"
    # 对应本地部署，则为lac.analysis_lexical(data=text, batch_size=1)
    data = {"text": text, "batch_size": 1}
    # 指定预测方法为lac并发送post请求，content-type类型应指定json方式
    url = "http://127.0.0.1:8866/predict/lac"
    # 指定post请求的headers为application/json方式
    headers = {"Content-Type": "application/json"}

    r = requests.post(url=url, headers=headers, data=json.dumps(data))

    # 打印预测结果
    print(json.dumps(r.json(), indent=4, ensure_ascii=False))


{
    "msg": "",
    "results": [
        {
            "tag": [
                "TIME",
                "v",
                "q",
                "n"
            ],
            "word": [
                "今天",
                "是",
                "个",
                "好日子"
            ]
        },
        {
            "tag": [
                "n",
                "v",
                "TIME",
                "v",
                "v"
            ],
            "word": [
                "天气预报",
                "说",
                "今天",
                "要",
                "下雨"
            ]
        }
    ],
    "status": "000"
}


In [ ]:
# 在终端关闭服务
# $ hub serving stop --port 8866


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 